<a href="https://colab.research.google.com/github/Mihan0207/Avatar_Fire_-_Ash_Prediction/blob/main/Avatar__Fire%26Ash_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Overvie - Avatar: Fire & Ash Box Office Predictor

The Goal To predict the Opening Week Revenue for the upcoming blockbuster ***Avatar: Fire and Ash (2025)*** using historical box office data from 2008–2024.

The Architecture We utilized a ***Star Schema***  warehousing approach:

**ETL Process**: We cleaned raw movie data, standardizing currencies and dates.

**Star Schema**: We split the data into a central Fact Table (financials) and peripheral Dimension Tables (Directors, Distributors) to organize the data efficiently.

**Feature Engineering**: We created advanced features like is_franchise .

The Model we employed is an Ensemble Machine Learning approach combining:

**Random Forest Regressor**: Good for handling non-linear relationships and interactions.

**Gradient Boosting Regressor**: Excellent for minimizing prediction errors iteratively.

The Result The model analyzes patterns from over 3,300 movies to forecast the financial performance of Avatar: Fire and Ash, considering factors like James Cameron's historical performance, the "Franchise Effect," and the massive impact of December holiday releases.


# Understanding the Fact Table

**What is it?**

The Fact Table is the central pillar of our "Star Schema" data model. It stores the quantitative / measurable data (the "facts") about the business events—in our case, the release of a movie.

**Why is it important?**

Centralization: It acts as the primary hub that connects all other data. Every row represents a specific event (a movie release).

Performance: By keeping text and descriptive details out of this table (storing only IDs and numbers), it remains lean and fast for calculations like "Sum of Total Gross" or "Average Opening Revenue."

**How it works in this project:**

Our fact table (fact_box_office) contains:

Metrics: total_gross, opening_revenue, budget (The numbers we want to analyze).

Foreign Keys: director_id, distributor_id (Links to the Dimension tables).

Degenerate Dimensions: movie_id, title (Identifiers kept here for simplicity).



### **Importing Essential Libraries**
This cell imports all the necessary Python libraries required for the movie data scraping and processing.
- `requests` and `BeautifulSoup`: Used for making HTTP requests to websites (like Box Office Mojo and IMDb) and parsing the HTML content, respectively. This forms the backbone of our web scraping efforts.
- `pandas`: A powerful library for data manipulation and analysis. It will be used to store, clean, and process the scraped movie data into a structured DataFrame format.
- `time` and `datetime`: Essential for handling time-related operations, such as adding delays between web requests to avoid being blocked, and parsing/formatting dates.
- `re` (Regular Expressions): Utilized for pattern matching in strings, especially for cleaning monetary values and extracting specific IDs from URLs.
- `random`: Used to introduce small, random delays in scraping to mimic human behavior and reduce the chance of detection by websites.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from datetime import datetime
import random

### **MovieDataScraper Class Definition and Methods**
This cell defines the `MovieDataScraper` class, which serves as the core engine for collecting movie data from various online sources. It encapsulates all the necessary logic and functions for the scraping process.

-   **`__init__(self, tmdb_api_key)`**: The constructor initializes the scraper with your TMDb API key, sets up base URLs for TMDb and Box Office Mojo, and configures HTTP headers to mimic a web browser. It also creates a `requests.Session` object for efficient, persistent connections during scraping.
-   **Helper Functions**: The class includes utility methods like `clean_money` for standardizing financial values extracted from web pages.
-   **Box Office Mojo Integration**: It contains methods (`get_yearly_releases`, `get_bo_metadata`, `get_opening_gross`) to fetch lists of movies, extract detailed metadata (budget, gross, release dates, distributor), and calculate opening weekend revenues from Box Office Mojo.
-   **External Data Integration**: Methods like `get_tmdb_data` (for TMDb details like ratings, genres, directors) and `get_imdb_rating` (for direct IMDb scraping) enrich the dataset with comprehensive external information.
-   **`run(self, start_year, end_year)` Orchestrator**: This central method coordinates the entire scraping process. It iterates through specified years, calls the various data fetching methods for each movie, merges the collected data, performs necessary cleanup, and saves the results into a pandas DataFrame, providing periodic backups.

This consolidated class design ensures that all scraping functionalities are logically grouped and easily manageable.

In [ ]:

class MovieDataScraper:
    def __init__(self, tmdb_api_key):
        self.tmdb_api_key = tmdb_api_key
        self.tmdb_base_url = "https://api.themoviedb.org/3"
        self.boxoffice_base_url = "https://www.boxofficemojo.com"
        # 🛡️ Browser Headers (Crucial for scraping IMDb/Mojo without getting blocked)
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept-Language': 'en-US,en;q=0.9',
        }
        self.session = requests.Session()

    # =========================================================================
    # 🛠️ HELPER FUNCTIONS
    # =========================================================================
    def clean_money(self, money_str):
        """Standardizes money strings ($1.5M -> 1500000.0)"""
        if not money_str: return None
        clean = re.sub(r'[^\d.]', '', money_str)
        try: return float(clean)
        except: return None

    # =========================================================================
    # 📌 STEP 1: BOX OFFICE MOJO (List & Metadata)
    # =========================================================================
    def get_yearly_releases(self, year):
        """Fetches the raw list of movies for a specific calendar year."""
        url = f"{self.boxoffice_base_url}/year/{year}/"
        try:
            response = self.session.get(url, headers=self.headers)
            if response.status_code != 200: return []
            soup = BeautifulSoup(response.content, 'html.parser')
            movies = []
            rows = soup.find_all('tr')[1:]
            for row in rows:
                cols = row.find_all('td')
                if len(cols) >= 2:
                    link = cols[1].find('a')
                    if link:
                        movies.append({'title': link.text.strip(), 'url': link['href']})
            return movies
        except: return []

    def get_bo_metadata(self, movie_url, title, original_year):
        """
        Extracts Budget, Total Gross, and CORRECTS the release year.
        Fix: Moves 'Alvin (2007)' out of the 2008 dataset if needed.
        """
        full_url = f"{self.boxoffice_base_url}{movie_url}"
        data = {
            'title': title, 'year': original_year, 'release_date': None,
            'total_gross': None, 'budget': 0, 'distributor': None,
            'fallback_weekend': None
        }
        try:
            resp = self.session.get(full_url, headers=self.headers)
            soup = BeautifulSoup(resp.content, 'html.parser')

            # 1. Money (Total Domestic)
            money = soup.find_all('span', class_='money')
            if money:
                data['total_gross'] = self.clean_money(money[0].text)
                if len(money) > 1: data['fallback_weekend'] = self.clean_money(money[0].text)

            # 2. Metadata (Budget & Year Fix)
            for div in soup.find_all('div', class_='a-section'):
                text = div.text.strip()
                if 'Budget' in text:
                    s = div.find('span', class_='money')
                    if s: data['budget'] = self.clean_money(s.text)
                elif 'Release Date' in text:
                    spans = div.find_all('span')
                    if len(spans) > 1:
                        date_str = spans[1].text.split('(')[0].strip()
                        data['release_date'] = date_str
                        # 🚨 YEAR FIX LOGIC
                        try:
                            dt = datetime.strptime(date_str, "%b %d, %Y")
                            data['year'] = dt.year # Overwrite year with actual release
                        except: pass
                elif 'Distributor' in text:
                    spans = div.find_all('span')
                    if len(spans) > 1: data['distributor'] = spans[1].text.strip()
            return data
        except: return None

    # =========================================================================
    # 📌 STEP 2: OPENING WEEK (Daily/Weekly Logic)
    # =========================================================================
    def get_opening_gross(self, movie_url):
        """Calculates 7-Day sum. Falls back to Weekly Table."""
        match = re.search(r'/release/(rl\d+)', movie_url)
        if not match: return None, "Error"
        rl_id = match.group(1)

        # Try DAILY (Sum 7 days)
        try:
            r = self.session.get(f"{self.boxoffice_base_url}/release/{rl_id}/daily/", headers=self.headers)
            if r.status_code == 200:
                rows = BeautifulSoup(r.content, 'html.parser').find_all('tr')[1:8] # First 7 days
                total = sum([self.clean_money(row.find_all('td')[3].text) for row in rows if '$' in row.text])
                if total > 0: return total, "7-Day Daily Sum"
        except: pass

        # Try WEEKLY (First row)
        try:
            r = self.session.get(f"{self.boxoffice_base_url}/release/{rl_id}/weekly/", headers=self.headers)
            if r.status_code == 200:
                row = BeautifulSoup(r.content, 'html.parser').find_all('tr')[1]
                val = self.clean_money(row.find_all('td')[2].text)
                if val: return val, "Weekly Table"
        except: pass

        return None, "Failed"

    # =========================================================================
    # 📌 STEP 3: EXTERNAL DATA (TMDb + IMDb)
    # =========================================================================
    def get_tmdb_data(self, title, year):
        """Get TMDb ID, Metadata, and External IDs"""
        try:
            # Search
            s_url = f"{self.tmdb_base_url}/search/movie"
            res = requests.get(s_url, params={'api_key': self.tmdb_api_key, 'query': title, 'year': year}).json()
            if not res['results']: return None

            tmdb_id = res['results'][0]['id']

            # Details
            d_url = f"{self.tmdb_base_url}/movie/{tmdb_id}"
            data = requests.get(d_url, params={'api_key': self.tmdb_api_key, 'append_to_response': 'credits,release_dates,external_ids'}).json()

            info = {
                'tmdb_id': tmdb_id,
                'imdb_id': data.get('external_ids', {}).get('imdb_id'),
                'tmdb_rating': data.get('vote_average'),
                'tmdb_votes': data.get('vote_count'),
                'tmdb_budget': data.get('budget', 0),
                'genres': ', '.join([g['name'] for g in data.get('genres', [])]),
                'runtime': data.get('runtime'),
                'director': next((c['name'] for c in data.get('credits', {}).get('crew', []) if c['job'] == 'Director'), None)
            }
            return info
        except: return None

    def get_imdb_rating(self, imdb_id):
        """
        Attempts to scrape IMDb directly.
        ⚠️ NOTE: IMDb is strict. If this fails/blocks, we return None.
        """
        if not imdb_id: return None, None
        url = f"https://www.imdb.com/title/{imdb_id}/"
        try:
            # We add a small random delay to look human
            time.sleep(random.uniform(0.1, 0.4))
            resp = self.session.get(url, headers=self.headers, timeout=5)

            if resp.status_code == 200:
                soup = BeautifulSoup(resp.content, 'html.parser')
                # IMDb 2024 Layout usually puts rating in specific data-testid or span
                # This is a generic robust finder for the rating (e.g., "8.1/10")
                rating_span = soup.find('div', {'data-testid': 'hero-rating-bar__aggregate-rating__score'})
                vote_div = soup.find('div', {'data-testid': 'hero-rating-bar__aggregate-rating__vote-count'})

                rating = float(rating_span.find('span').text) if rating_span else None

                votes = None
                if vote_div:
                    # Clean vote text (e.g. "2.1M" -> 2100000)
                    v_text = vote_div.find_all('span')[0].text.strip()
                    v_text = v_text.replace('K', '000').replace('M', '000000').replace('.', '')
                    votes = int(v_text)

                return rating, votes
        except:
            return None, None
        return None, None

    # =========================================================================
    # 🚀 ORCHESTRATOR
    # =========================================================================
    def run(self, start_year, end_year):
        all_data = []
        print(f"🎬 STARTED: {start_year}-{end_year}")

        for year in range(start_year, end_year + 1):
            movies = self.get_yearly_releases(year)
            print(f"\n📅 Processing Year {year} ({len(movies)} movies found)...")

            for i, m in enumerate(movies):
                # 1. Get BOM Metadata (Budget, Year, Gross)
                bo = self.get_bo_metadata(m['url'], m['title'], year)
                if not bo or not bo['total_gross']: continue

                # 2. Get Opening Week (Daily Sum or Weekly)
                open_w, source = self.get_opening_gross(m['url'])
                if not open_w:
                    if bo['fallback_weekend']:
                        open_w = bo['fallback_weekend']
                        source = "Weekend Fallback"
                    else: continue

                # 3. Get TMDb Data
                tmdb = self.get_tmdb_data(m['title'], bo['year']) # Use corrected year
                if not tmdb: continue

                # 4. Get IMDb Data (Optional Scrape)
                imdb_rating, imdb_votes = self.get_imdb_rating(tmdb['imdb_id'])

                # 5. Merge
                row = {**bo, **tmdb}
                row['opening_revenue'] = open_w
                row['opening_source'] = source
                row['final_budget'] = max(bo['budget'] or 0, tmdb['tmdb_budget'] or 0)
                row['imdb_rating'] = imdb_rating
                row['imdb_votes'] = imdb_votes

                # If IMDb scrape failed, use TMDb as proxy?
                # Better to keep distinct columns so you know the source.

                # Cleanup
                for k in ['budget', 'tmdb_budget', 'fallback_weekend']: row.pop(k, None)

                all_data.append(row)
                print(f"   ✅ {m['title'][:25]:<25} | 7Day: ${open_w:,.0f} | IMDb: {imdb_rating or 'N/A'}")

            # Save Backup
            pd.DataFrame(all_data).to_csv(f"movie_data_{start_year}_{end_year}.csv", index=False)

        return pd.DataFrame(all_data)

# --- EXECUTE ---
if __name__ == "__main__":
    key = input("Enter TMDb Key: ")
    scraper = MovieDataScraper(key)
    df = scraper.run(2008, 2024)
    print("Done!")

Enter TMDb Key: 14f60f8dd81401f5f867c1b1913d4749
🎬 STARTED: 2008-2024

📅 Processing Year 2008 (200 movies found)...
   ✅ The Dark Knight           | 7Day: $238,615,211 | IMDb: 9.1
   ✅ Iron Man                  | 7Day: $3,500,000 | IMDb: 7.9
   ✅ Indiana Jones and the Kin | 7Day: $25,041,072 | IMDb: 6.2
   ✅ Hancock                   | 7Day: $41,273,567 | IMDb: 6.4
   ✅ WALL·E                    | 7Day: $94,686,825 | IMDb: 8.4
   ✅ Kung Fu Panda             | 7Day: $83,677,338 | IMDb: 7.6
   ✅ Madagascar: Escape 2 Afri | 7Day: $81,887,894 | IMDb: 6.7
   ✅ Twilight                  | 7Day: $93,362,477 | IMDb: 5.4
   ✅ Quantum of Solace         | 7Day: $82,082,841 | IMDb: 6.5
   ✅ Horton Hears a Who!       | 7Day: $61,419,921 | IMDb: 6.8
   ✅ Sex and the City          | 7Day: $77,958,978 | IMDb: 5.7
   ✅ Mamma Mia!                | 7Day: $44,848,740 | IMDb: 6.5
   ✅ The Chronicles of Narnia: | 7Day: $68,045,010 | IMDb: 6.5
   ✅ The Incredible Hulk       | 7Day: $74,919,370 | IMDb: 6.6
  

# Understanding Dimension Tables

**What are they?**

Dimension Tables provide the "Context" (the "Who, What, Where") for the numbers in the Fact Table. They store descriptive attributes that we use to filter, group, and label our data.

**Why are they important?**

Descriptive Power: They hold the rich text data (e.g., "Christopher Nolan", "Action", "Warner Bros.").

Normalization: Instead of repeating the text "Steven Spielberg" 30 times in the Fact Table (which wastes space and risks spelling errors), we store it once in the Director Dimension with ID 101. The Fact Table just points to 101.

**How it works in this project:**

dim_director: Stores director_name, total_movies, and their avg_gross track record.

dim_distributor: Stores distributor_name and their market_share.

Usage: When the model needs to know if a Director is profitable, it looks up their ID in the Fact Table, jumps to the dim_director table, and retrieves their historical avg_gross.

### **DimensionTable creation and Methods**
This cell defines the `EnhancedDimensionTableCreator` class, which is central to transforming raw movie data into a structured star schema for analytical purposes. It encapsulates the entire process of data cleaning, dimension table creation, and saving the results.

-   **`__init__(self, fact_table_path)`**: The constructor loads the raw `movie_data_2008_2024.csv` into a pandas DataFrame. It then performs crucial in-place data cleaning steps:
    -   Standardizing `distributor` names by removing boilerplate text and converting 'nan' strings to `None`.
    -   Cleaning and converting `release_date` strings to datetime objects, handling various formats and `NaN` values.
    -   Filling `NaN` values in all numeric columns (`final_budget`, `total_gross`, `opening_revenue`, `tmdb_rating`, `tmdb_votes`, `imdb_rating`) with `0` to prepare for aggregations.
-   **`_clean_date(self, date_str)`**: A private helper function used by `__init__` to parse and clean date strings robustly.
-   **Dimension Creation Methods**: The class includes dedicated methods for creating each dimension table:
    -   `create_movie_dimension()`: Generates a table with unique movie identifiers and descriptive attributes like title, year, runtime, genres, and an 'era' categorization.
    -   `create_people_dimension()`: Extracts unique directors and cast members, assigns their roles, and creates a unique `person_id` for each.
    -   `create_director_dimension()`: Calculates aggregate performance metrics for each director (e.g., total movies, average gross) and assigns a `director_id`.
    -   `create_genre_dimension()`: Identifies unique genres, categorizes them into broader types, and assigns a `genre_id`.
    -   `create_distributor_dimension()`: Summarizes each distributor's performance with metrics like total movies, total gross, and market share, assigning a `distributor_id`.
    -   `create_date_dimension()`: Creates a comprehensive table of unique release dates with attributes like year, month, day, quarter, and a weekend flag.
-   **Helper Functions (`get_era`, `categorize_genre`)**: These are utility functions used internally by dimension creation methods to derive new categorical features.
-   **`create_all_dimensions(self, output_dir)`**: This orchestrator method calls all individual dimension creation methods, saves the resulting DataFrames as CSV files in a specified output directory, and provides console feedback on the process. It ensures a systematic and organized output of the star schema components.

In [ ]:
import pandas as pd
import numpy as np
import re
import os

class EnhancedDimensionTableCreator:
    def __init__(self, fact_table_path):
        """
        Load the Fact Table (CSV) and prepare it for Dimension extraction.
        """
        self.df = pd.read_csv(fact_table_path)
        print(f"📂 Loaded {len(self.df)} records from {fact_table_path}")

        # --- DATA CLEANING (In-Place for Dimension Extraction) ---

        # 1. Clean Distributor names
        if 'distributor' in self.df.columns:
            self.df['distributor'] = self.df['distributor'].astype(str).str.replace('See full company information', '', regex=False).str.strip()
            self.df['distributor'] = self.df['distributor'].replace({'nan': None, 'None': None})

        # 2. Clean Release Date
        if 'release_date' in self.df.columns:
            self.df['release_date'] = self.df['release_date'].astype(str).apply(self._clean_date)
            self.df['release_date'] = pd.to_datetime(self.df['release_date'], errors='coerce')

        # 3. Fill numeric NaNs for safe aggregation
        numeric_cols = ['final_budget', 'total_gross', 'opening_revenue', 'tmdb_rating', 'tmdb_votes', 'imdb_rating']
        for col in numeric_cols:
            if col in self.df.columns:
                self.df[col] = pd.to_numeric(self.df[col], errors='coerce').fillna(0)

    def _clean_date(self, date_str):
        if pd.isna(date_str) or str(date_str).lower() == 'nan': return None
        return re.split(r'[
-]', str(date_str))[0].strip()

    # =========================================================================
    # 1. MOVIE DIMENSION
    # =========================================================================
    def create_movie_dimension(self):
        print("  creating Movie Dimension...")
        # Define columns to include from your Fact Table
        desired_cols = [
            'title', 'year', 'runtime', 'distributor',
            'tmdb_id', 'imdb_id', 'release_date',
            'genres', 'tmdb_rating', 'imdb_rating',
            'tagline', 'overview'
        ]

        # Create Movie ID if not present
        if 'movie_id' not in self.df.columns:
            self.df['movie_id'] = range(1, len(self.df) + 1)

        # Select available columns
        available_cols = ['movie_id'] + [c for c in desired_cols if c in self.df.columns]
        movie_dim = self.df[available_cols].copy()

        # Add Era
        movie_dim['era'] = movie_dim['year'].apply(self.get_era)

        return movie_dim

    # =========================================================================
    # 2. PEOPLE DIMENSION (Director & Cast)
    # =========================================================================
    def create_people_dimension(self):
        print("  creating People Dimension...")
        all_people = {}

        # Directors
        if 'director' in self.df.columns:
            for d in self.df['director'].dropna().unique():
                d = str(d).strip()
                if d:
                    if d not in all_people: all_people[d] = {'name': d, 'roles': set()}
                    all_people[d]['roles'].add('Director')

        # Cast
        if 'cast' in self.df.columns:
            for cast_str in self.df['cast'].dropna():
                actors = [x.strip() for x in str(cast_str).split(',')]
                for a in actors:
                    if a:
                        if a not in all_people: all_people[a] = {'name': a, 'roles': set()}
                        all_people[a]['roles'].add('Actor')

        # DataFrame
        people_list = []
        for i, (name, data) in enumerate(all_people.items(), 1):
            people_list.append({
                'person_id': i,
                'name': name,
                'roles': ', '.join(sorted(data['roles']))
            })

        return pd.DataFrame(people_list)

    # =========================================================================
    # 3. DIRECTOR DIMENSION
    # =========================================================================
    def create_director_dimension(self):
        print("  creating Director Dimension...")
        if 'director' not in self.df.columns: return pd.DataFrame()

        stats = []
        for d in self.df['director'].dropna().unique():
            subset = self.df[self.df['director'] == d]

            # Using YOUR column names for aggregation
            stats.append({
                'director_name': d,
                'total_movies': len(subset),
                'avg_total_gross': subset['total_gross'].mean(),
                'avg_opening_revenue': subset['opening_revenue'].mean(),
                'avg_final_budget': subset['final_budget'].mean(),
                'avg_tmdb_rating': subset['tmdb_rating'].mean()
            })

        df_dir = pd.DataFrame(stats)
        if not df_dir.empty:
            df_dir['director_id'] = range(1, len(df_dir)+1)
        return df_dir

    # =========================================================================
    # 4. GENRE DIMENSION
    # =========================================================================
    def create_genre_dimension(self):
        print("  creating Genre Dimension...")
        if 'genres' not in self.df.columns: return pd.DataFrame()

        unique_genres = set()
        for g_str in self.df['genres'].dropna():
            parts = [x.strip() for x in str(g_str).split(',')]
            unique_genres.update(parts)

        df_gen = pd.DataFrame({'genre_id': range(1, len(unique_genres)+1), 'genre_name': sorted(list(unique_genres))})
        df_gen['category'] = df_gen['genre_name'].apply(self.categorize_genre)
        return df_gen

    # =========================================================================
    # 5. DISTRIBUTOR DIMENSION
    # =========================================================================
    def create_distributor_dimension(self):
        print("  creating Distributor Dimension...")
        if 'distributor' not in self.df.columns: return pd.DataFrame()

        stats = []
        for d in self.df['distributor'].dropna().unique():
            subset = self.df[self.df['distributor'] == d]

            stats.append({
                'distributor_name': d,
                'total_movies': len(subset),
                'total_gross': subset['total_gross'].sum(),
                'avg_opening_revenue': subset['opening_revenue'].mean(),
                'avg_final_budget': subset['final_budget'].mean()
            })

        df_dist = pd.DataFrame(stats)
        if not df_dist.empty:
            total_market = df_dist['total_gross'].sum()
            df_dist['market_share'] = (df_dist['total_gross'] / total_market * 100).round(2) if total_market > 0 else 0
            df_dist['distributor_id'] = range(1, len(df_dist)+1)

        return df_dist

    # =========================================================================
    # 6. DATE DIMENSION
    # =========================================================================
    def create_date_dimension(self):
        print("  creating Date Dimension...")
        if 'release_date' not in self.df.columns: return pd.DataFrame()

        dates = pd.to_datetime(self.df['release_date']).dropna().unique()
        data = []
        for i, d in enumerate(sorted(dates), 1):
            data.append({
                'date_id': i,
                'full_date': d,
                'year': d.year,
                'month': d.month,
                'month_name': d.strftime('%B'),
                'day': d.day,
                'quarter': d.quarter,
                'is_weekend': 1 if d.weekday() >= 5 else 0
            })

        return pd.DataFrame(data)

    # --- HELPERS ---
    def get_era(self, year):
        if year < 2010: return 'Pre-2010'
        elif year < 2015: return '2010-2014'
        elif year < 2020: return '2015-2019'
        else: return 'Post-2020'

    def categorize_genre(self, genre):
        if genre in ['Action', 'Adventure', 'Thriller', 'Science Fiction']: return 'Action/Sci-Fi'
        elif genre in ['Drama', 'Romance', 'History']: return 'Drama'
        elif genre in ['Comedy', 'Family', 'Animation']: return 'Comedy/Family'
        else: return 'Other'

    def create_all_dimensions(self, output_dir='dimensions'):
        os.makedirs(output_dir, exist_ok=True)

        dims = {
            'dim_movie': self.create_movie_dimension(),
            'dim_people': self.create_people_dimension(),
            'dim_director': self.create_director_dimension(),
            'dim_genre': self.create_genre_dimension(),
            'dim_distributor': self.create_distributor_dimension(),
            'dim_date': self.create_date_dimension()
        }

        print("\n📊 Dimensions Created Successfully!")
        for name, df in dims.items():
            if not df.empty:
                save_path = f"{output_dir}/{name}.csv"
                df.to_csv(save_path, index=False)
                print(f"   ✅ {name}: {len(df)} rows saved to {save_path}")
            else:
                print(f"   ⚠️ {name}: Skipped (No Data)")

        return dims

### **Main Execution Block for Dimension Table Creator**
This final cell contains the `if __name__ == "__main__":` block, which serves as the entry point for running the `EnhancedDimensionTableCreator`. It coordinates the entire process of transforming the raw movie data into a structured star schema.

-   **Fact Table Path**: It first defines `FACT_TABLE_PATH` as 'movie_data_2008_2024.csv'. This is the expected output file from the preceding `MovieDataScraper` step, containing all the collected movie data that will serve as our central 'fact' table.
-   **File Existence Check**: Before proceeding, the code checks if the specified `FACT_TABLE_PATH` file actually exists in the current working directory. This is a critical validation step to ensure the data source is available.
-   **Initialization and Dimension Creation**:
    -   If the fact table is found, an instance of the `EnhancedDimensionTableCreator` class is created, passing the path to the fact table. This initializes the class, loading and performing initial cleaning on the data.
    -   Subsequently, the `creator.create_all_dimensions()` method is called. This method orchestrates the generation of all the individual dimension tables (Movie, People, Director, Genre, Distributor, Date) and saves them as separate CSV files within a 'dimensions' folder.
-   **Error Handling and User Guidance**: If the `FACT_TABLE_PATH` file is not found, an informative error message is printed. This message guides the user to verify that the previous data scraping step was successful and that the output file (`movie_data_2008_2024.csv`) is present in the correct location, highlighting the dependency between the scraping and dimension creation processes.

In [ ]:
# ==========================================
# 🚀 MAIN EXECUTION
# ==========================================
if __name__ == "__main__":
    # Ensure this matches your file name exactly
    FACT_TABLE_PATH = 'movie_data_2008_2024.csv'

    if os.path.exists(FACT_TABLE_PATH):
        print(f"🚀 Reading Fact Table from: {FACT_TABLE_PATH}")
        creator = EnhancedDimensionTableCreator(FACT_TABLE_PATH)
        creator.create_all_dimensions()
        print("\n✅ DONE! check the 'dimensions' folder.")
    else:
        print(f"❌ Error: '{FACT_TABLE_PATH}' not found. Please ensure the file is in the same folder.")